## Q-learning

This notebook will guide you through implementation of vanilla Q-learning algorithm.

You need to implement QLearningAgent (follow instructions for each method) and use it on a number of tests below.

In [ ]:
#XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1
        
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Try it on taxi

Here we use the qlearning agent on taxi env from openai gym.
You will need to insert a few agent functions here.

In [ ]:
import gym
env = gym.make("Taxi-v2")

n_actions = env.action_space.n
print(f'Number of actions: {n_actions}')

In [ ]:
from qlearning import QLearningAgent

agent = QLearningAgent(alpha=0.5, 
                       epsilon=0.25, 
                       discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

In [ ]:
def play_and_train(env,
                   agent,
                   t_max=10**4):
    """
    Run the full game, with actions given by agent's epsilon-greedy policy
    and updates the policy whenever possible
    
    Parameters
    ----------
    env : gym-object
        The environment to play with
    agent : QLearningAgent
        The agent to play and train with
    t_max : int
        The maximum number of steps to take
    
    Returns
    -------
    total_reward : float
        The accumulated reward
    """
    total_reward = 0.0
    s = env.reset()
    
    for t in range(t_max):
        # get agent to pick action given state s.
        a = agent.get_action(s)
        
        next_s, r, done, _ = env.step(a)
        
        # train (update) agent for state s
        agent.update(s, a, r, next_s)
        
        s = next_s
        total_reward +=r
        
        if done: 
            break
        
    return total_reward

In [ ]:
from IPython.display import clear_output

rewards = []
for i in range(1000):
    rewards.append(play_and_train(env, agent))
    agent.epsilon *= 0.99
    
    if i %100 ==0:
        clear_output(True)
        print('eps =', agent.epsilon, 'mean reward =', np.mean(rewards[-10:]))
        plt.plot(rewards)
        plt.show()

### Submit to Coursera I: Preparation

In [ ]:
submit_rewards1 = rewards.copy()

# Binarized state spaces

Use agent to train efficiently on CartPole-v0.
This environment has a continuous set of possible states, so you will have to group them into bins somehow.

The simplest way is to use `round(x,n_digits)` (or numpy round) to round real number to a given amount of digits.

The tricky part is to get the n_digits right for each state to train effectively.

Note that you don't need to convert state to integers, but to __tuples__ of any kind of values.

In [ ]:
env = gym.make("CartPole-v0")
n_actions = env.action_space.n

print("first state:%s" % (env.reset()))
plt.imshow(env.render('rgb_array'))

### Play a few games

We need to estimate observation distributions. To do so, we'll play a few games and record all states.

In [ ]:
all_states = []
for _ in range(1000):
    all_states.append(env.reset())
    done = False
    while not done:
        s, r, done, _ = env.step(env.action_space.sample())
        all_states.append(s)
        if done: 
            break
            
all_states = np.array(all_states)

# NOTE: There are four observations for each state
#       cart position, cart velocity, pole angle and pole velocity at tip
#       https://github.com/openai/gym/wiki/CartPole-v0#observation
#       Here we plot the distribution of each one by one
for obs_i in range(env.observation_space.shape[0]):
    plt.hist(all_states[:, obs_i], bins=20)
    plt.show()

## Binarize environment

In [ ]:
from gym.core import ObservationWrapper

class Binarizer(ObservationWrapper):
    """
    Class to put observations into bins
    """
    
    def _observation(self, state):    
        """
        Will round the states to binarize them
        
        Based on the plots above, we binarize
        cart_position: 1 digits
        cart_velocity: 1 digit
        pole_angle: 2 digits
        pole_velocity_at_tip : 2 digit
        
        Parameters
        ----------
        state : tuple
            The tuple of states
        
        Returns
        -------
        tuple
            The binarized states
        """
        
        cart_position, cart_velocity, pole_angle, pole_velocity_at_tip = state
        cart_position = round(cart_position, 1)
        cart_velocity = round(cart_velocity, 1)
        pole_angle = round(pole_angle, 2)
        pole_velocity_at_tip = round(pole_velocity_at_tip, 0)
        state = (cart_position, cart_velocity, pole_angle, pole_velocity_at_tip)

        return tuple(state)

In [ ]:
env = Binarizer(gym.make("CartPole-v0"))

In [ ]:
all_states = []
for _ in range(1000):
    all_states.append(env.reset())
    done = False
    while not done:
        s, r, done, _ = env.step(env.action_space.sample())
        all_states.append(s)
        if done: break
            
all_states = np.array(all_states)

for obs_i in range(env.observation_space.shape[0]):
    
    plt.hist(all_states[:,obs_i],bins=20)
    plt.show()

## Learn binarized policy

Now let's train a policy that uses binarized state space.

__Tips:__ 
* If your binarization is too coarse, your agent may fail to find optimal policy. In that case, change binarization. 
* If your binarization is too fine-grained, your agent will take much longer than 1000 steps to converge. You can either increase number of iterations and decrease epsilon decay or change binarization.
* Having 10^3 ~ 10^4 distinct states is recommended (`len(QLearningAgent._qvalues)`), but not required.


In [ ]:
agent = QLearningAgent(alpha=0.5, epsilon=0.25, discount=0.99,
                       get_legal_actions = lambda s: range(n_actions))

In [ ]:
rewards = []
for i in range(10000):
    rewards.append(play_and_train(env, agent))   
    
    #OPTIONAL YOUR CODE: adjust epsilon
    if i % 20 ==0:
        # Decrease epsilon to increase exploitation
        agent.epsilon *= 0.995
        clear_output(True)
        print(f'eps = {agent.epsilon:.4}, mean reward = {np.mean(rewards[-10:])}')
        plt.plot(rewards)
        plt.show()

In [ ]:
print(len(agent._qvalues))

### Submit to Coursera II: Submission

In [ ]:
EMAIL = ''
TOKEN = ''

In [ ]:
submit_rewards2 = rewards.copy()
from submit import submit_qlearning_all
submit_qlearning_all(submit_rewards1, submit_rewards2, EMAIL, TOKEN)